In [ ]:
!pip install keras_vggface keras_applications mtcnn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 4.7 MB/s 
     |████████████████████████████████| 2.3 MB 14.9 MB/s 


In [ ]:
import os
os.listdir('drive/MyDrive/missing child model/images')

['Sachin', 'Virat', 'Obama']

In [ ]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from numpy import asarray
from mtcnn.mtcnn import MTCNN
import sys

In [ ]:
# extract a single face from an uploaded photograph
detector = MTCNN()
def extract_face(filename,detector=detector,required_size=(224, 224)):
	pixels = plt.imread(filename)
	# create the detector, using default weights
	# detect faces in the image
	results = detector.detect_faces(pixels)
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
	return face_array, image
    # resize pixels to the model size


In [ ]:
img_folder_path = 'drive/MyDrive/missing child model/images'
for folder in os.listdir(img_folder_path):
    images = os.listdir(os.path.join(img_folder_path, folder))
    
    for image in images:
      print(f"{img_folder_path}/{folder}/{image}")
      face_arr, face_img = extract_face(f"{img_folder_path}/{folder}/{image}")
      face_img.save(f"{img_folder_path}/{folder}/{image}")

drive/MyDrive/missing child model/images/Sachin/sachin.jpg
1/1 [==============================] - 0s 137ms/step
drive/MyDrive/missing child model/images/Virat/Virat.jpg
1/1 [==============================] - 0s 26ms/step
drive/MyDrive/missing child model/images/Obama/obama.jpg
1/1 [==============================] - 0s 28ms/step


# Build model

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

train_generator = \
    train_datagen.flow_from_directory(
img_folder_path,
target_size=(224,224),
color_mode='rgb',
batch_size=32,
class_mode='categorical',
shuffle=True)

Found 3 images belonging to 3 classes.


In [ ]:
train_generator.class_indices.values()
# dict_values([0, 1, 2])
NO_CLASSES = len(train_generator.class_indices.values())
NO_CLASSES

3

In [ ]:
filename = "/usr/local/lib/python3.7/dist-packages/keras_vggface/models.py"
text = open(filename).read()
open(filename, "w+").write(text.replace('keras.engine.topology', 'tensorflow.keras.utils'))

from keras_vggface.vggface import VGGFace

base_model = VGGFace(include_top=True,
    model='vgg16',
    input_shape=(224, 224, 3))
base_model.summary()

print(len(base_model.layers))
# 26 layers in the original VGG-Face

580070376/580070376 [==============================] - 18s 0us/step
Model: "vggface_vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_1 (Conv2D)            (None, 224, 224, 64)      1792      
                                                                 
 conv1_2 (Conv2D)            (None, 224, 224, 64)      36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 64)      0         
                                                                 
 conv2_1 (Conv2D)            (None, 112, 112, 128)     73856     
                                                                 
 conv2_2 (Conv2D)            (None, 112, 112, 128)     147584    
                                                   

In [ ]:
base_model = VGGFace(include_top=False,
model='vgg16',
input_shape=(224, 224, 3))
base_model.summary()
print(len(base_model.layers))
# 19 layers after excluding the last few layers

58909280/58909280 [==============================] - 2s 0us/step
Model: "vggface_vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_1 (Conv2D)            (None, 224, 224, 64)      1792      
                                                                 
 conv1_2 (Conv2D)            (None, 224, 224, 64)      36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 64)      0         
                                                                 
 conv2_1 (Conv2D)            (None, 112, 112, 128)     73856     
                                                                 
 conv2_2 (Conv2D)            (None, 112, 112, 128)     147584    
                                                      

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)

# final layer with softmax activation
x = Dense(NO_CLASSES, activation='softmax')(x)
transfer_model = Model(inputs = base_model.input, outputs = x)

In [ ]:
print(transfer_model.summary())
print(len(transfer_model.layers))


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_1 (Conv2D)            (None, 224, 224, 64)      1792      
                                                                 
 conv1_2 (Conv2D)            (None, 224, 224, 64)      36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 64)      0         
                                                                 
 conv2_1 (Conv2D)            (None, 112, 112, 128)     73856     
                                                                 
 conv2_2 (Conv2D)            (None, 112, 112, 128)     147584    
                                                                 
 pool2 (MaxPooling2D)        (None, 56, 56, 128)       0   

In [ ]:
# don't train the first 19 layers - 0..18
for layer in transfer_model.layers[:19]:
    layer.trainable = False

# train the rest of the layers - 19 onwards
for layer in transfer_model.layers[19:]:
    layer.trainable = True


In [ ]:
transfer_model.compile(optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])


In [ ]:
transfer_model.fit(train_generator,
  batch_size = 1,
  verbose = 1,
  epochs = 100)

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 1.0969 - accuracy: 0.3333
Epoch 2/100
1/1 [==============================] - 1s 1s/step - loss: 0.9529 - accuracy: 1.0000
Epoch 3/100
1/1 [==============================] - 1s 1s/step - loss: 0.7804 - accuracy: 1.0000
Epoch 4/100
1/1 [==============================] - 1s 1s/step - loss: 0.5737 - accuracy: 1.0000
Epoch 5/100
1/1 [==============================] - 1s 1s/step - loss: 0.3563 - accuracy: 1.0000
Epoch 6/100
1/1 [==============================] - 1s 1s/step - loss: 0.1760 - accuracy: 1.0000
Epoch 7/100
1/1 [==============================] - 1s 1s/step - loss: 0.0678 - accuracy: 1.0000
Epoch 8/100
1/1 [==============================] - 1s 1s/step - loss: 0.0209 - accuracy: 1.0000
Epoch 9/100
1/1 [==============================] - 1s 1s/step - loss: 0.0056 - accuracy: 1.0000
Epoch 10/100
1/1 [==============================] - 1s 1s/step - loss: 0.0014 - accuracy: 1.0000
Epoch 11/100
1/1 [=====================

In [ ]:
# creates a HDF5 file
transfer_model.save('drive/MyDrive/missing child model/transfer_learning_trained_face_cnn_model.h5')

In [ ]:
from tensorflow.keras.models import load_model

# deletes the existing model
del transfer_model

# returns a compiled model identical to the previous one
model = load_model('drive/MyDrive/missing child model/transfer_learning_trained_face_cnn_model.h5')

In [ ]:
print(model.summary())
len(model.layers)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_1 (Conv2D)            (None, 224, 224, 64)      1792      
                                                                 
 conv1_2 (Conv2D)            (None, 224, 224, 64)      36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 64)      0         
                                                                 
 conv2_1 (Conv2D)            (None, 112, 112, 128)     73856     
                                                                 
 conv2_2 (Conv2D)            (None, 112, 112, 128)     147584    
                                                                 
 pool2 (MaxPooling2D)        (None, 56, 56, 128)       0   

24